In [1]:
from PIL import Image
from os import listdir
from os.path import isdir
from numpy import asarray, expand_dims
import numpy as np
import os


In [2]:
!unzip faces.zip


Archive:  faces.zip
   creating: faces/
   creating: faces/train/
   creating: faces/train/deiski/
  inflating: faces/train/deiski/AGGT2684.JPG  
  inflating: faces/train/deiski/AGGT2684_flip.JPG  
  inflating: faces/train/deiski/AGIJ8108.JPG  
  inflating: faces/train/deiski/AGIJ8108_flip.JPG  
  inflating: faces/train/deiski/AHAD5612.JPG  
  inflating: faces/train/deiski/AHAD5612_flip.JPG  
  inflating: faces/train/deiski/ANFX1598.JPG  
  inflating: faces/train/deiski/ANFX1598_flip.JPG  
  inflating: faces/train/deiski/ANJC2104.JPG  
  inflating: faces/train/deiski/ANJC2104_flip.JPG  
  inflating: faces/train/deiski/ASKJ7426.JPG  
  inflating: faces/train/deiski/ASKJ7426_flip.JPG  
  inflating: faces/train/deiski/AVSY9849.JPG  
  inflating: faces/train/deiski/AVSY9849_flip.JPG  
  inflating: faces/train/deiski/BGRD9737.JPG  
  inflating: faces/train/deiski/BGRD9737_flip.JPG  
  inflating: faces/train/deiski/BHYM9667.JPG  
  inflating: faces/train/deiski/BHYM9667_flip.JPG  
  inflatin

Passa o path do arquivo, abre o arquivo como imagem, converte em RGB e em numpy, e retorna.

In [3]:
def load_face(filename):
  
  # carregando imagem do arquivo
  image = Image.open(filename)
  
  #converter para RGB
  #image = Image.convert("RGB")

  return asarray(image)


In [4]:
# Carregando as faces de um diretório
def load_faces(directory_src):
  
  faces = list()

  #iterando arquivos "directory_src = nome da pasta com o nome da pessoa"
  for filename in listdir(directory_src): 
    
    path = directory_src + filename 
    
    #a leitura da imagem pode dar erro então usa try except
    try:
      faces.append(load_face(path))
    except:
      print("Erro na imagem {}".format(path))

  return faces


In [5]:
## carregando todo o dataset de imagens de faces

def load_fotos(directory_src):
  
  X, y = list(), list()

  #iterar as pastas por classes "directory_src = é a pasta faces"
  for subdir in listdir(directory_src):
    #path
    path = directory_src + subdir + '/'

    if not isdir(path):
      continue
    
    faces = load_faces(path)

    labels = [subdir for _ in range(len(faces))]

    # sumarizar progresso
    print('>Carregadas %d faces da classe: %s' % (len(faces), subdir))

    X.extend(faces)
    y.extend(labels)

  
  return asarray(X), asarray(y)

In [23]:
import matplotlib.pyplot as plt

trainX, trainy = load_fotos(directory_src="/content/faces/train/") #para gerar embeddings das faces de treinamento

## após terminar de gerar o .csv (embedding) das faces de treinamento, gerar embeddings para faces de validação também
#trainX, trainy = load_fotos(directory_src="/content/faces/validation/") #para gerar embeddings das faces de validação

print(trainX[0])

>Carregadas 14 faces da classe: deiski
>Carregadas 57 faces da classe: nikoru
[[[104 105  99]
  [109 110 104]
  [ 48  48  46]
  ...
  [180 146 118]
  [255 243 208]
  [250 255 215]]

 [[ 91  92  84]
  [ 96  97  91]
  [ 78  79  74]
  ...
  [227 196 167]
  [255 252 217]
  [251 255 219]]

 [[ 86  86  78]
  [ 47  47  39]
  [ 67  68  62]
  ...
  [255 230 199]
  [255 254 220]
  [251 255 223]]

 ...

 [[  7   7   7]
  [  7   7   7]
  [  6   8   7]
  ...
  [  8   8   8]
  [  7   7   7]
  [  8   8   8]]

 [[  7   9   8]
  [  6   8   7]
  [  5   7   6]
  ...
  [  7   7   7]
  [  6   6   6]
  [  6   6   6]]

 [[  5   6   8]
  [  5   7   6]
  [  7   9   8]
  ...
  [  7   7   5]
  [  6   6   4]
  [  8   8   8]]]


In [24]:
trainX.shape

(71, 160, 160, 3)

In [25]:
trainy.shape

(71,)

In [10]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
print(tf.__version__) 

model = load_model('facenet_keras.h5')


2.6.0


In [11]:
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

## **FUNÇÃO GERADORA DE EMBEDDINGS**




In [12]:
def get_embedding(model, face_pixels):
  
    # PADRONIZAÇÃO
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std

    # TRANSFORMAR A FACE EM 1 UNICO EXEMPLO
    # (320, 320) -> (1, 320, 320)
    samples = expand_dims(face_pixels, axis=0)
    
    
    # REALIZAR A PREDIÇÃO GERANDO O EMBEDDING
    yhat = model.predict(samples)   ##[[1,2..128]... [1,2..128] ]
    
    return yhat[0]

In [26]:
newTrainX = list()

In [27]:
#Debug para verificar se não está misturado imagens que não são (160, 160) de resolução
for face in trainX:
      print(face.shape)
      if face.shape != (160, 160, 3):
          plt.imshow(face)
          plt.title('Imagem convertida para o RGB')
          plt.show()

(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)


In [28]:
for face in trainX:
    embedding = get_embedding(model, face)
    newTrainX.append(embedding) # Terá uma lista de embeddings

In [29]:
newTrainX = asarray(newTrainX)
print(newTrainX.shape)

(71, 128)


In [30]:
import pandas as pd

df = pd.DataFrame(data=newTrainX)

In [31]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
0,-0.206436,-0.746464,0.278254,-0.587223,0.025434,0.848655,-0.226615,0.132980,1.182832,-0.282673,0.773295,-0.117908,1.079490,-0.519124,-0.629820,-0.161693,0.195582,-0.199930,0.603669,0.464424,-0.158680,0.212851,-0.532143,1.155205,0.732386,0.665504,0.710068,0.070197,-0.139090,-0.634856,0.454976,0.155202,0.137783,-0.165073,-0.178813,-0.067420,-0.159426,0.487659,0.443822,0.062338,...,-0.352302,-0.086140,-0.670040,-0.563643,-0.072814,0.444570,0.761420,0.296646,-0.332085,0.235130,-0.450460,-0.563659,1.149487,-0.093320,0.589385,-0.573453,-0.646295,0.485657,-0.400344,-0.230257,-0.258993,-1.281964,1.002152,0.097178,0.665930,-0.165873,0.045478,0.354351,-0.487647,0.653829,0.031314,0.743142,0.469287,-0.155531,0.340291,0.506537,0.315118,-0.140103,0.569573,0.195775
1,-0.130806,-0.575447,0.159703,-0.330744,0.291311,0.709786,-0.105924,0.070527,0.473588,0.160885,0.414585,-0.316870,0.664935,-0.650055,-0.374357,-0.003153,0.108312,0.254863,0.162665,-0.255638,0.101090,0.210526,-0.082747,0.449387,0.472441,0.274098,0.765621,0.332784,-0.602351,0.149052,0.126633,0.613915,-0.078197,-0.028018,0.452732,-0.064221,-0.587907,0.233957,0.123147,0.584642,...,-0.600441,0.312378,-0.063927,0.327874,-0.229674,0.296439,0.710958,0.331260,0.044558,-0.152363,-0.089675,0.034717,0.654028,0.177147,0.252963,-0.506898,-0.402632,0.245751,-1.027006,-0.183330,-0.488793,-1.096510,0.768343,-0.216323,0.238540,0.470417,0.239898,0.525333,-0.383017,0.557899,0.341589,0.498575,-0.007220,-0.178509,0.289650,0.481818,0.023550,-0.335846,0.319535,0.132350
2,0.080751,-0.162013,-0.268597,-0.055308,1.056408,-1.207288,-0.314635,0.087923,0.858524,-0.991308,0.418723,-0.712151,-0.483829,-0.903477,0.036907,0.960611,0.180463,0.638263,-1.078037,-0.690597,-0.739978,1.905965,-0.992930,0.723715,-0.567212,0.337273,0.026981,0.562894,0.010235,0.231316,-0.045646,0.733602,-0.216198,0.326769,0.482446,0.403070,-0.802598,-0.576467,0.227998,0.460948,...,0.135177,1.087766,0.762439,0.919384,-0.201308,-0.500199,0.117675,-0.241030,-0.090686,0.240880,-0.730144,-0.457665,-0.290217,0.507112,0.496674,-0.651408,0.181769,0.228390,-0.201277,0.467379,-0.871299,-0.558181,0.858459,1.086597,0.595151,0.465922,1.299041,-0.228225,0.658769,0.193363,-0.037036,0.318306,-0.965410,-0.557025,0.398265,0.250854,-0.136564,-0.160151,-1.028355,0.061811
3,-0.230243,-0.999213,0.284826,-1.235806,0.183500,0.162685,0.154523,0.596713,1.469765,-0.483936,0.473132,0.027018,-0.035792,-0.519663,-0.183437,-0.081853,-0.362870,0.582098,0.364931,-0.894860,0.221756,0.355561,-0.128156,0.789784,0.680087,0.345221,0.954721,-0.296160,0.194049,0.080085,-0.395161,0.980086,0.057079,0.365536,-0.009111,-0.231252,-0.407675,-0.158127,-0.449051,-0.292848,...,0.084862,0.466409,-0.166569,-0.011511,-0.523260,0.035937,0.913964,0.689441,-0.926657,0.203140,-0.148813,-0.389825,1.156706,-0.468938,0.581379,-1.306757,-0.096355,0.675187,-0.396087,0.023806,0.342760,-1.080936,0.466083,0.275048,0.347622,0.245121,0.522914,0.147650,-0.047130,0.337729,0.907266,0.938855,0.468463,0.042699,0.446190,-0.086410,0.848373,-0.711378,0.419533,0.379627
4,0.294907,-0.873148,-0.071881,-1.667662,0.171365,-0.390323,-0.263469,0.333539,0.616051,-0.673439,0.680274,0.484820,-0.680117,-0.661052,0.158783,0.147006,-0.233894,0.321984,0.205010,-1.200472,-0.239211,0.173318,-0.126704,0.573103,0.739806,0.153657,0.595125,0.425735,0.504453,-0.125043,-0.151342,0.768122,-0.085424,-0.000506,-0.060603,-0.087503,-0.510712,0.783086,0.258351,-0.243803,...,0.302381,0.138759,0.050356,-0.571389,-0.393735,-0.183200,1.119267,0.187382,-0.925499,-0.196728,0.228010,-0.415111,0.480452,-0.118153,0.890241,-0.882063,-0.018629,0.446168,0.209224,0.522467,0.361774,-0.766514,0.606461,0.552802,0.510295,0.067353,0.713876,0.253705,0.111962,1.111428,0.419513,0.254362,0.473169,0.4

In [32]:
df['target'] = trainy

In [33]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,target
0,-0.206436,-0.746464,0.278254,-0.587223,0.025434,0.848655,-0.226615,0.132980,1.182832,-0.282673,0.773295,-0.117908,1.079490,-0.519124,-0.629820,-0.161693,0.195582,-0.199930,0.603669,0.464424,-0.158680,0.212851,-0.532143,1.155205,0.732386,0.665504,0.710068,0.070197,-0.139090,-0.634856,0.454976,0.155202,0.137783,-0.165073,-0.178813,-0.067420,-0.159426,0.487659,0.443822,0.062338,...,-0.086140,-0.670040,-0.563643,-0.072814,0.444570,0.761420,0.296646,-0.332085,0.235130,-0.450460,-0.563659,1.149487,-0.093320,0.589385,-0.573453,-0.646295,0.485657,-0.400344,-0.230257,-0.258993,-1.281964,1.002152,0.097178,0.665930,-0.165873,0.045478,0.354351,-0.487647,0.653829,0.031314,0.743142,0.469287,-0.155531,0.340291,0.506537,0.315118,-0.140103,0.569573,0.195775,deiski
1,-0.130806,-0.575447,0.159703,-0.330744,0.291311,0.709786,-0.105924,0.070527,0.473588,0.160885,0.414585,-0.316870,0.664935,-0.650055,-0.374357,-0.003153,0.108312,0.254863,0.162665,-0.255638,0.101090,0.210526,-0.082747,0.449387,0.472441,0.274098,0.765621,0.332784,-0.602351,0.149052,0.126633,0.613915,-0.078197,-0.028018,0.452732,-0.064221,-0.587907,0.233957,0.123147,0.584642,...,0.312378,-0.063927,0.327874,-0.229674,0.296439,0.710958,0.331260,0.044558,-0.152363,-0.089675,0.034717,0.654028,0.177147,0.252963,-0.506898,-0.402632,0.245751,-1.027006,-0.183330,-0.488793,-1.096510,0.768343,-0.216323,0.238540,0.470417,0.239898,0.525333,-0.383017,0.557899,0.341589,0.498575,-0.007220,-0.178509,0.289650,0.481818,0.023550,-0.335846,0.319535,0.132350,deiski
2,0.080751,-0.162013,-0.268597,-0.055308,1.056408,-1.207288,-0.314635,0.087923,0.858524,-0.991308,0.418723,-0.712151,-0.483829,-0.903477,0.036907,0.960611,0.180463,0.638263,-1.078037,-0.690597,-0.739978,1.905965,-0.992930,0.723715,-0.567212,0.337273,0.026981,0.562894,0.010235,0.231316,-0.045646,0.733602,-0.216198,0.326769,0.482446,0.403070,-0.802598,-0.576467,0.227998,0.460948,...,1.087766,0.762439,0.919384,-0.201308,-0.500199,0.117675,-0.241030,-0.090686,0.240880,-0.730144,-0.457665,-0.290217,0.507112,0.496674,-0.651408,0.181769,0.228390,-0.201277,0.467379,-0.871299,-0.558181,0.858459,1.086597,0.595151,0.465922,1.299041,-0.228225,0.658769,0.193363,-0.037036,0.318306,-0.965410,-0.557025,0.398265,0.250854,-0.136564,-0.160151,-1.028355,0.061811,deiski
3,-0.230243,-0.999213,0.284826,-1.235806,0.183500,0.162685,0.154523,0.596713,1.469765,-0.483936,0.473132,0.027018,-0.035792,-0.519663,-0.183437,-0.081853,-0.362870,0.582098,0.364931,-0.894860,0.221756,0.355561,-0.128156,0.789784,0.680087,0.345221,0.954721,-0.296160,0.194049,0.080085,-0.395161,0.980086,0.057079,0.365536,-0.009111,-0.231252,-0.407675,-0.158127,-0.449051,-0.292848,...,0.466409,-0.166569,-0.011511,-0.523260,0.035937,0.913964,0.689441,-0.926657,0.203140,-0.148813,-0.389825,1.156706,-0.468938,0.581379,-1.306757,-0.096355,0.675187,-0.396087,0.023806,0.342760,-1.080936,0.466083,0.275048,0.347622,0.245121,0.522914,0.147650,-0.047130,0.337729,0.907266,0.938855,0.468463,0.042699,0.446190,-0.086410,0.848373,-0.711378,0.419533,0.379627,deiski
4,0.294907,-0.873148,-0.071881,-1.667662,0.171365,-0.390323,-0.263469,0.333539,0.616051,-0.673439,0.680274,0.484820,-0.680117,-0.661052,0.158783,0.147006,-0.233894,0.321984,0.205010,-1.200472,-0.239211,0.173318,-0.126704,0.573103,0.739806,0.153657,0.595125,0.425735,0.504453,-0.125043,-0.151342,0.768122,-0.085424,-0.000506,-0.060603,-0.087503,-0.510712,0.783086,0.258351,-0.243803,...,0.138759,0.050356,-0.571389,-0.393735,-0.183200,1.119267,0.187382,-0.925499,-0.196728,0.228010,-0.415111,0.480452,-0.118153,0.890241,-0.882063,-0.018629,0.446168,0.209224,0.522467,0.361774,-0.766514,0.606461,0.552802,0.510295,0.067353,0.713876,0.253705,0.111962,1.111428,0.419513,0.254362,0.473169,0.452594,0.819543,

In [34]:
df.to_csv('faces.csv', index=False)